In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# 應用Keras-rl tensorflow 2.0 版在Gym環境學習CartPole策略

In [0]:
!pip install tensorflow==2.0.0-beta > /dev/null 2>&1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow.compat.v2 as tf
import random
import numpy as np
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
tf.enable_v2_behavior()
print(tf.__version__)

2.0.0-beta0


In [0]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

## 模擬隨機行為: 左或右

In [0]:
from random import randrange
def action_my():
  return(randrange(2))

# Wrap env 纔可以在Colab上看Cartpole的模擬影片

In [0]:
env = wrap_env(gym.make('CartPole-v1'))
np.random.seed(123)
env.seed(123)

total_timesteps = 0
for i_episode in range(5):
    observation = env.reset()
    for t in range(500):
        env.render()
        action = action_my()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            total_timesteps += (t+1)
            break

print('Average score over {} random strategy games: {}'.format(i_episode+1, total_timesteps/(i_episode+1)))
env.close()

Episode finished after 14 timesteps
Episode finished after 11 timesteps
Episode finished after 17 timesteps
Episode finished after 31 timesteps
Episode finished after 19 timesteps
Average score over 5 random strategy games: 18.4


In [0]:
show_video()

## 安裝Keras-rl tensorflow 2.0 beta0版

In [0]:
!pip install keras-rl2 > /dev/null 2>&1

# 若要重新訓練，跑這以後的程式區塊即可 ...

In [0]:
env = gym.make('CartPole-v1')
np.random.seed(123)
env.seed(123)

[123]

## 定義神經網路的架構，輸入參數分別是狀態的維度及可執行的行動數量

In [0]:
def agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions, activation='linear'))
#    model.add(Dense(actions, activation='sigmoid'))
    return model
  
model = agent(env.observation_space.shape[0], env.action_space.n)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                160       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 66        
Total params: 2,338
Trainable params: 2,338
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
from rl.agents import SARSAAgent
from rl.policy import EpsGreedyQPolicy, BoltzmannQPolicy

policy = BoltzmannQPolicy()

In [0]:
sarsa = SARSAAgent(model = model, policy = policy, nb_actions = env.action_space.n)

In [0]:
sarsa.compile('adam', metrics = ['mse'])

In [0]:
sarsa.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: 1.0000
272 episodes - episode_reward: 36.195 [8.000, 336.000] - loss: 10.072 - mse: 598.187 - mean_q: 31.893

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: 1.0000
73 episodes - episode_reward: 135.356 [13.000, 500.000] - loss: 6.984 - mse: 1589.110 - mean_q: 55.406

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 63s 6ms/step - reward: 1.0000
45 episodes - episode_reward: 220.111 [9.000, 500.000] - loss: 7.624 - mse: 2228.989 - mean_q: 64.974

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 62s 6ms/step - reward: 1.0000
33 episodes - episode_reward: 311.939 [10.000, 500.000] - loss: 9.416 - mse: 2697.714 - mean_q: 73.600

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 63s 6ms/step - reward: 1.0000
d

In [0]:
scores = sarsa.test(env, nb_episodes = 50, visualize= False)
print('Average score over 50 test games:{}'.format(np.mean(scores.history['episode_reward'])))

Testing for 50 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 500.000, steps: 500
Episode 25: reward: 5

In [0]:
#sarsa.save_weights('sarsa_weights.h5f', overwrite=True)

In [0]:
# sarsa.load_weights('sarsa_weights.h5f')

In [0]:
_ = sarsa.test(env, nb_episodes = 2, visualize= True)

Testing for 2 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500


# 定義行動函數應用已訓練好的model，輸入參數為狀態向量

In [0]:
def action_sarsa(observation):
  # observation是環境回傳的狀態，在此的shape是(4,)要改為
  # (1,1,4)才可當作是model的輸入
  # 於此輸入改shape的程式碼


  next_action_reference = model.predict(observation)
  # next_action_reference的維度即是可執行的行動數量
  # 在此是2，是含有兩筆實數的List
  # 於此輸入將next_action_reference向量轉為行動的程式碼

  return(next_action)

env = wrap_env(gym.make('CartPole-v1'))

total_timesteps = 0
for i_episode in range(5):
    observation = env.reset()
    for t in range(500):
        env.render()
        action = action_sarsa(observation)
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            total_timesteps += (t+1)
            break

print('Average score over 50 test games:{}'.format(total_timesteps/(i_episode+1)))
env.close()  



Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps
Average score over 50 test games:500.0


In [0]:
show_video()